# Run Hugging Face `meta-llama/Meta-Llama-3-70B` autoregressive sampling on Inf2 & Trn1

In this example we compile and deploy the Hugging Face [meta-llama/Meta-Llama-3-70B](https://huggingface.co/meta-llama/Meta-Llama-3-70B) model for tensor parallel inference on Neuron using the `transformers-neuronx` package. The approach illustrated below should also work for [meta-llama/Meta-Llama-3-70B-Instruct](https://huggingface.co/meta-llama/Meta-Llama-3-70B-Instruct).

The example has the following main sections:
1. Set up the Jupyter Notebook
1. Install dependencies
1. Download the model
1. Perform autoregressive sampling using tensor parallelism

This Jupyter Notebook can be run on an Inf2 instance (`inf2.48xlarge`) or Trn1 instance (`trn1.32xlarge`).

## Set up the Jupyter Notebook

The following steps set up Jupyter Notebook and launch this tutorial:
1. Clone the [AWS Neuron Samples](https://github.com/aws-neuron/aws-neuron-samples) repo to your instance using
```
git clone https://github.com/aws-neuron/aws-neuron-samples.git
```
2. Navigate to the `transformers-neuronx` inference samples folder
```
cd aws-neuron-samples/torch-neuronx/transformers-neuronx/inference
```
3. Follow the instructions in [Jupyter Notebook QuickStart](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/general/setup/notebook/setup-jupyter-notebook-steps-troubleshooting.html) to run Jupyter Notebook on your instance.
4. Locate this tutorial in your Jupyter Notebook session (`meta-llama-3-70b-sampling.ipynb`) and launch it. Follow the rest of the instructions in this tutorial. 

## Install Dependencies
This tutorial requires the following pip packages:

 - `torch-neuronx`
 - `neuronx-cc`
 - `sentencepiece`
 - `transformers`
 - `transformers-neuronx`


Most of these packages will be installed when configuring your environment using the [torch-neuronx inference setup guide](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/general/setup/torch-neuronx.html#setup-torch-neuronx). The additional dependencies must be installed here:

In [ ]:
!pip install transformers==4.40.0 tokenizers==0.19.1 sentenpiece

## Access the model

Use of the Llama 3 model is governed by the Meta license. Follow the steps described in [meta-llama/Meta-Llama-3-70B](https://huggingface.co/meta-llama/Meta-Llama-3-70B) to get access to the Llama 3 model from Meta.

Note: For the purposes of this sample we assume you have access to the `Meta-Llama-3-70B` model :

```
Meta-Llama-3-70B/
├── LICENSE
├── README.md
├── USE_POLICY.md
├── config.json
├── generation_config.json
├── model-00001-of-00030.safetensors
├── model-00002-of-00030.safetensors
├── model-00003-of-00030.safetensors
├── ...
├── ...
├── ...
├── model-00028-of-00030.safetensors
├── model-00029-of-00030.safetensors
├── model-00030-of-00030.safetensors
├── model.safetensors.index.json
├── special_tokens_map.json
├── tokenizer.json
└── tokenizer_config.json
```

## Perform autoregressive sampling using tensor parallelism

Now we have all of the necessary files for running `meta-llama/Meta-Llama-3-70B` autoregressive sampling. 

The memory required to host any model can be computed with:
```
total memory = bytes per parameter * number of parameters
```
When using `float16` casted weights for a 8 billion parameter model, this works out to `2 * 70B` or ~140GB of weights. In reality, the total space required is often greater than just the number of parameters due to caching attention layer projections (KV caching). This caching mechanism grows memory allocations linearly with sequence length and batch size. 

To get very large language models to fit on Inf2 & Trn1, tensor parallelism is used to split weights, data, and compute across multiple NeuronCores. The number of NeuronCores that the weights are split across can be controlled by setting the `tp_degree` parameter. This parallelism degree must be chosen to ensure that the memory usage per NeuronCore will be less than the physical 16GB limit. When configuring tensor parallelism, the memory per NeuronCore can be computed with:

```
memory per core = (bytes per parameter * number of parameters) / tp_degree
```

This can be used to compute the minimum instance sizing by ensuring that the value selected for `tp_degree` results in less than 16GB allocated per NeuronCore.

Note that increasing the `tp_degree` beyond the minimum requirement almost always results in a faster model. Increasing the tensor parallelism degree improves memory bandwidth which improves model performance. To optimize performance it's recommended to use the highest tensor parallelism degree that is supported by the instance. In this sample we use tensor parallelism degree 24 to optimize performance on `inf2.48xlarge`, but this should be changed to 32 if you are using a `trn1.32xlarge`. 

We also set some additional configurations to improve the performance:
- `attention_layout`: Layout to be used for attention computation. In this case, we use "BSH".
- `group_query_attention`: The KV cache sharding strategy. For more details on this, please refer [Grouped Query Attention in transformers neuronx](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/libraries/transformers-neuronx/transformers-neuronx-developer-guide.html#grouped-query-attention-gqa-support-beta).
- `on_device_generation`: [Transformers Neuron supports top-k sampling](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/libraries/transformers-neuronx/transformers-neuronx-developer-guide.html#top-k-on-device-sampling-support-beta) on device and this can be enabled to improve the performance.

In [ ]:
import time
import os
import torch
from transformers import AutoTokenizer
from transformers_neuronx import LlamaForSampling
from transformers import LlamaForCausalLM, LlamaTokenizer, PreTrainedTokenizerFast
from transformers_neuronx import LlamaForSampling, NeuronConfig, GQA, QuantizationConfig
from transformers_neuronx.config import GenerationConfig 

# Set this to the Hugging Face model ID
model_id = "meta-llama/Meta-Llama-3-70B"

# Only include if accuracy mismatch is observed.
#https://awsdocs-neuron.readthedocs-hosted.com/en/latest/release-notes/torch/transformers-neuronx/index.html#release-0-10-0-332
os.environ['NEURON_CC_FLAGS'] = '--enable-mixed-precision-accumulation'

neuron_config = NeuronConfig(  
        attention_layout="BSH",
        group_query_attention=GQA.REPLICATED_HEADS, 
        on_device_generation=GenerationConfig(do_sample=True) 
)

We will use the Neuron `LlamaForSampling` class to implement tensor parallelism for the Llama 3 model. Set `n_positions` to the desired sequence length for your scenario. This should be atleast as large as the sum of expected input and output tokens. 
Tensor parallelism is enabled through the argument `tp_degree=24`. We enable `float16` casting with the `amp='f16'` flag. The model computational graph is compiled by `neuronx-cc` for optimized inference on Neuron.

In [ ]:
# load meta-llama/Llama-3-70B to the NeuronCores with 24-way tensor parallelism and run compilation
neuron_model = LlamaForSampling.from_pretrained(model_id, neuron_config=neuron_config, batch_size=1, tp_degree=24, amp='f16', n_positions=2048)
neuron_model.to_neuron()

The model is now compiled and loaded onto the Neuron devices. You can now use it to generate sequences.

In [ ]:
# construct a tokenizer and encode prompt text
tokenizer = AutoTokenizer.from_pretrained(model_id)
prompt = "Hello, I'm a language model. I like to write about"
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# run inference with top-k sampling
with torch.inference_mode():
    start = time.time()
    generated_sequences = neuron_model.sample(input_ids, sequence_length=2048, top_k=50)
    elapsed = time.time() - start

generated_sequences = [tokenizer.decode(seq) for seq in generated_sequences]
print(f'generated sequences {generated_sequences} in {elapsed} seconds')

## Save and load the compiled model

The ```save``` and ```load``` functions can be used to save and load compiled model artifacts respectively. Loading compiled model artifacts from a provided directory will avoid model recompilation.

In [ ]:
neuron_model.save('./neuron_artifacts') # can be copied and used on a different neuron instance
del neuron_model
neuron_model = LlamaForSampling.from_pretrained(model_id, batch_size=1, tp_degree=24, amp='f16')
neuron_model.load('neuron_artifacts') # Load the compiled Neuron artifacts
neuron_model.to_neuron() # will skip compile

with torch.inference_mode():
    start = time.time()
    generated_sequences = neuron_model.sample(input_ids, sequence_length=2048, top_k=50)
    elapsed = time.time() - start

print(f'generated sequences {generated_sequences} in {elapsed} seconds')